In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'e:\\TextSummarization'

In [19]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class datatransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [20]:
from textSummarizer.utils.common import read_yaml, create_directories
from textSummarizer.constants import *

In [25]:
class ConfigurationManager:
    
    def __init__(
        self,
        config_path = CONFIG_FILE_PATH,
        params_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_datatransformation_config(self) -> datatransformationConfig:
        
        config = self.config.data_transformation
        
        create_directories([config.root_dir])
        datatransformation_config = datatransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        
        return datatransformation_config

In [26]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [35]:
class data_transformation:
    def __init__(self, config: datatransformationConfig):
        
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
        
    def convert(self, batch):
        
        input_encodings = self.tokenizer(batch['dialogue'], max_length=1024, truncation=True)
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(batch['summary'], max_length=128, truncation=True)
            
        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
        
    def create_dataset(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [36]:
config = ConfigurationManager()
data_transformation_config = config.get_datatransformation_config()
data_transformation = data_transformation(config=data_transformation_config)
data_transformation.create_dataset()

[2023-07-09 15:24:05,210: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-09 15:24:05,213: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-09 15:24:05,216: INFO: common: directory: artifacts created successfully]
[2023-07-09 15:24:05,218: INFO: common: directory: artifacts/data_transformation created successfully]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\hp\.conda\envs\textS\lib\site-packages\transformers\tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
